# README

This notebook gives an example of how to run the GEM-PRO pipeline on a **SBML model**.

## Installation

See: https://github.com/nmih/ssbio/blob/master/README.md
- If something isn't working, make sure to update the repository before you do anything (git pull)

## Quick start

I just want to get structures ASAP! How can I do that?

- my_gempro.run_pipeline() -- in progress!

In [1]:
# Import the GEM-PRO class
from ssbio.pipeline.gempro import GEMPRO

In [2]:
# Other imports for this example
import os
import pandas as pd
import os.path as op

In [3]:
# Printing multiple outputs per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
# Create logger
import logging
logger = logging.getLogger()

############# SET YOUR LOGGING LEVEL HERE #############
# - CRITICAL
#     - Only really important messages shown
# - ERROR
#     - Major errors
# - WARNING
#     - Warnings that don't affect running of the pipeline
# - INFO
#     - Info such as the number of structures mapped per gene
# - DEBUG
#     - Really detailed information that will print out a lot of stuff
logger.setLevel(logging.INFO)
#######################################################

## Initialization of the project

Set these three things:

- GEM_NAME
    - Your project name
- ROOT_DIR
    - The directory where the GEM_NAME folder will be created
- GENES
    - Your list of gene IDs
    
A directory will be created in ROOT_DIR named after your GEM_NAME. The folders are:
```
    .
    ├── data  # where dataframes are stored
    ├── figures  # where figures are stored
    ├── model  # where SBML and GEM-PRO models are stored
    ├── notebooks  # location of any ipython notebooks for analyses
    ├── sequences  # sequences are stored here, in gene specific folders
    │   ├── <gene_id1>
    │   │   └── sequence.fasta
    │   ├── <gene_id2>
    │   └── ...
    └── structures
        ├── by_gene  # structures are stored here, in gene specific folders
        │   ├── <gene_id1>
        │   │   └── 1abc.pdb
        │   ├── <gene_id2>
        │   └── ...
        └── by_complex  # complexes for reactions are stored here (in progress)
```

In [5]:
GEM_NAME = 'mtuberculosis_gp_atlas'

ROOT_DIR = '/home/nathan/projects_unsynced/'

GEM_FILE = '/home/nathan/projects_unsynced/mtuberculosis_gp_atlas/model/iNJ661.json'

In [6]:
# Create the GEM-PRO project
# Specify that the model is in json format
my_gempro = GEMPRO(gem_name=GEM_NAME, root_dir=ROOT_DIR, gem_file_path=GEM_FILE, gem_file_type='json')

INFO:ssbio.pipeline.gempro:Loaded model: /home/nathan/projects_unsynced/mtuberculosis_gp_atlas/model/iNJ661.json
INFO:ssbio.pipeline.gempro:Number of reactions: 1025
INFO:ssbio.pipeline.gempro:Number of reactions linked to a gene: 720
INFO:ssbio.pipeline.gempro:Number of genes (excluding spontaneous): 661
INFO:ssbio.pipeline.gempro:Number of metabolites: 826


## Mapping gene ID to sequence

**First, we want to set a sequence for each of the genes in our model.** This can happen one of two ways:

1. I want to map the gene IDs in the model (I don't have the original genome sequence)
2. I have the sequences for my organism

---------------------

### 1. I want to map the gene IDs in the model


##### Mapping with KEGG

- Files created:
    - Per gene, the KEGG sequence and metadata is downloaded in the GEM-PRO "sequences" folder.
    

- Usage:
        my_gempro.kegg_mapping_and_metadata(kegg_organism_code, custom_gene_mapping=None, force_rerun=False)


- Arguments:

    - *kegg_organism_code*
        - See the full list of organisms here: http://www.genome.jp/kegg/catalog/org_list.html
        - E. coli MG1655 is "eco"
        - M. tuberculosis is "mtu"

    - *custom_gene_mapping*
        - If the model gene IDs differ from the KEGG gene IDs, and you know the mapping, supply it as a dictionary here. 
        - An example would be for the *T. maritima* model, where the model IDs are formatted without an underscore ("TM0001") while in KEGG they have an underscore ("TM_0001").

    - *force_rerun*
        - If you want to force the rerun of mapping, set this to True.


- Creates attributes:
    - A summary of the metadata is available in the "df_kegg_metadata" attribute.
            my_gempro.df_kegg_metadata
            
    - Any gene IDs that are missing a mapping are reported in the "missing_kegg_mapping" attribute.
            my_gempro.missing_kegg_mapping
        
    
##### Mapping with UniProt

- Method:
    - You can try mapping your genes using the actual service here: http://www.uniprot.org/uploadlists/
    
    
- Files created:
    - Per gene, the sequence and metadata is downloaded in the GEM-PRO "sequences" folder.
    

- Usage:
        my_gempro.uniprot_mapping_and_metadata(model_gene_source, custom_gene_mapping=None, force_rerun=False)
        
        
- Arguments:

    - *model_gene_source*
        - Here is a list of the gene IDs that can be mapped to UniProt IDs: http://www.uniprot.org/help/programmatic_access#id_mapping_examples
        - *E. coli* b-numbers are of the source "ENSEMBLGENOME_ID"
        - *M. tuberculosis* gene IDs match the source "TUBERCULIST_ID"
        
    - *custom_gene_mapping*
        - If you know the model gene ID to UniProt ID mapping, supply it as a dictionary here.
        
    - *force_rerun*
        - If you want to force the rerun of mapping, set this to True.
        
        
- Creates attributes:
    - A summary of the metadata is available in the "df_uniprot_metadata" attribute.
            my_gempro.df_uniprot_metadata
            
    - Any gene IDs that are missing a mapping are reported in the "missing_uniprot_mapping" attribute.
            my_gempro.missing_uniprot_mapping
        
            
##### Consolidating information

If you have mapped with both KEGG and UniProt mappers, then you can set a representative sequence for the gene using this function.

- Method:
    - Manual mappings override all existing mappings.
    - UniProt mappings override KEGG mappings except when KEGG mappings have PDBs associated with them and UniProt doesn't.


- Usage:
        my_gempro.set_representative_sequence()


- Creates attributes:
    - A summary of the mappings available in the "df_sequence_mapping" attribute.
            my_gempro.df_sequence_mapping
            
    - Any gene IDs that are missing a mapping are reported in the "missing_mapping" attribute.
            my_gempro.missing_mapping
        

---------------------------


### 2. I have the sequences for my organism

If you already have the amino acid sequences for each gene, simply set them as the representative sequence using the below function.

##### Setting the amino acid sequences

- Method:
    - This is to be used when you actually have the genome source as protein coding sequences, or are for some reason interested in a random collection of sequences.
    - These sequences are set as "representative" for each gene. That means you wouldn't have any KEGG or UniProt ID set as representative though. If you do run the above KEGG or UniProt mappings, it will check if they match your sequence and add in metadata.

- Usage:
        my_gempro.manual_seq_mapping(gene_to_seq_dict)
        
        
- Arguments:

    - *gene_to_seq_dict*
        - Supply a dictionary mapping the model gene IDs to their amino acid sequences.
        - Example: 
                {'Rv1295': 'MTVPPTATHQPWPGVIAAYRDRLPVGDDWTPVTLLEGGTPLIAATNLSK'}

-------------------

### Gene annotations

Check out the gene annotations saved directly into the gene. 

- **These are COBRApy Gene objects! If you started with a SBML model, these are directly annotated within the model.**

- Usage:
        my_gempro.genes.get_by_id('Rv1295').annotation
        my_gempro.genes.get_by_id('Rv1295').annotation['sequence']

In [7]:
# Example of manual_seq_mapping
gene_to_seq_dict = {'Rv1295': 'MTVPPTATHQPWPGVIAAYRDRLPVGDDWTPVTLLEGGTPLIAATNLSKQTGCTIHLKVEGLNPTGSFKDRGMTMAVTDALAHGQRAVLCASTGNTSASAAAYAARAGITCAVLIPQGKIAMGKLAQAVMHGAKIIQIDGNFDDCLELARKMAADFPTISLVNSVNPVRIEGQKTAAFEIVDVLGTAPDVHALPVGNAGNITAYWKGYTEYHQLGLIDKLPRMLGTQAAGAAPLVLGEPVSHPETIATAIRIGSPASWTSAVEAQQQSKGRFLAASDEEILAAYHLVARVEGVFVEPASAASIAGLLKAIDDGWVARGSTVVCTVTGNGLKDPDTALKDMPSVSPVPVDPVAVVEKLGLA'}

my_gempro.manual_seq_mapping(gene_to_seq_dict)

INFO:ssbio.pipeline.gempro:Loaded in 1 sequences


In [8]:
my_gempro.genes.get_by_id('Rv1295').annotation['sequence']['representative']

{'kegg_id': None,
 'metadata_file': None,
 'pdbs': [],
 'properties': {},
 'seq_file': 'Rv1295.faa',
 'seq_len': 360,
 'uniprot_acc': None}

In [9]:
# KEGG mapping of gene ids
my_gempro.kegg_mapping_and_metadata(kegg_organism_code='mtu')
my_gempro.df_kegg_metadata.head(2)
my_gempro.missing_kegg_mapping

Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"
INFO:ssbio.pipeline.gempro:Created KEGG metadata dataframe. See the "df_kegg_metadata" attribute.


,gene,uniprot_acc,kegg_id,seq_len,pdbs,seq_file,metadata_file
0,Rv0417,NaN,NaN,252,[],mtu-Rv0417.faa,mtu-Rv0417.kegg
1,Rv2291,NaN,NaN,284,[],mtu-Rv2291.faa,mtu-Rv2291.kegg


['Rv0417',
 'Rv2291',
 'Rv3737',
 'Rv1295',
 'Rv1559',
 'Rv3113',
 'Rv2447c',
 'Rv1201c',
 'Rv2713',
 'Rv3464',
 'Rv3468c',
 'Rv3784',
 'Rv3266c',
 'Rv3465',
 'Rv1672c',
 'Rv1448c',
 'Rv1739c',
 'Rv2397c',
 'Rv2399c',
 'Rv2398c',
 'Rv2400c',
 'Rv0951',
 'Rv0952',
 'Rv0248c',
 'Rv3316',
 'Rv3319',
 'Rv3317',
 'Rv0247c',
 'Rv3318',
 'Rv2443',
 'Rv0553',
 'Rv0542c',
 'Rv0234c',
 'Rv1731',
 'Rv3846',
 'Rv0432',
 'Rv1908c',
 'Rv2601',
 'Rv1200',
 'Rv1079',
 'Rv0391',
 'Rv2539c',
 'Rv2552c',
 'Rv2847c',
 'Rv0555',
 'Rv1704c',
 'Rv0069c',
 'Rv2335',
 'Rv1464',
 'Rv0858c',
 'Rv1202',
 'Rv2386c',
 'Rv1285',
 'Rv1286',
 'Rv2958c',
 'Rv2962c',
 'Rv2465c',
 'Rv1408',
 'Rv1981c',
 'Rv0570',
 'Rv3048c',
 'Rv3051c',
 'Rv1237',
 'Rv1236',
 'Rv1238',
 'Rv2436',
 'Rv1416',
 'Rv1412',
 'Rv2786c',
 'Rv2848c',
 'Rv2064',
 'Rv1594',
 'Rv3777',
 'Rv1902c',
 'Rv0162c',
 'Rv1617',
 'Rv2607',
 'Rv3307',
 'Rv2965c',
 'Rv0408',
 'Rv0436c',
 'Rv3042c',
 'Rv0505c',
 'Rv0884c',
 'Rv0437c',
 'Rv3227',
 'Rv1017c',
 'R

In [10]:
# UniProt mapping of gene_id -> uniprot_id
my_gempro.uniprot_mapping_and_metadata(model_gene_source='TUBERCULIST_ID')
my_gempro.df_uniprot_metadata.head(2)

INFO:root:getUserAgent: Begin
INFO:root:getUserAgent: user_agent: EBI-Sample-Client/ (services.py; Python 3.5.2; Linux) Python-requests/2.11.1
INFO:root:getUserAgent: End
Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"
INFO:ssbio.pipeline.gempro:Created UniProt metadata dataframe. See the "df_uniprot_metadata" attribute.


,gene,uniprot_acc,seq_len,seq_file,pdbs,gene_name,reviewed,kegg_id,refseq,ec_number,pfam,description,entry_version,seq_version,metadata_file
0,Rv0417,P9WG73,252.0,P9WG73.fasta,[],thiG,True,[mtu:Rv0417],"[NP_214931.1, NC_000962.3, WP_003916659.1, NZ_...",[2.8.1.10],NaN,[Thiazole synthase {ECO:0000255|HAMAP-Rule:MF_...,2016-10-05,2014-04-16,P9WG73.txt
1,Rv2291,P9WHF5,284.0,P9WHF5.fasta,[],sseB,True,[mtu:Rv2291],"[NP_216807.1, NC_000962.3, WP_003899253.1, NZ_...",[2.8.1.1],[PF00581],[Putative thiosulfate sulfurtransferase SseB],2016-10-05,2014-04-16,P9WHF5.txt


In [11]:
# Manually adding in UniProt mappings later
manual_uniprot = pd.read_csv(op.join(my_gempro.data_dir, '161019-gene_to_uniprot.in'))
manual_uniprot_dict = {}
for i,r in manual_uniprot.iterrows():
    manual_uniprot_dict[r[0]] = r[1]
print(manual_uniprot_dict)
my_gempro.manual_uniprot_mapping(manual_uniprot_dict)

INFO:ssbio.pipeline.gempro:Updated existing UniProt dataframe.


{'Rv1755c': 'P9WIA9', 'Rv2321c': 'P71891', 'Rv2322c': 'P71890', 'Rv0619': 'Q79FY3', 'Rv0618': 'Q79FY4'}


In [12]:
# Consolidate mappings sources
my_gempro.set_representative_sequence()
my_gempro.df_sequence_mapping.head(2)
my_gempro.missing_mapping

INFO:ssbio.pipeline.gempro:Created sequence mapping dataframe. See the "df_sequence_mapping" attribute.


,gene,uniprot_acc,kegg_id,pdbs,seq_len,seq_file,metadata_file
0,Rv0417,P9WG73,mtu:Rv0417,NaN,252.0,P9WG73.fasta,P9WG73.txt
1,Rv2291,P9WHF5,mtu:Rv2291,NaN,284.0,P9WHF5.fasta,P9WHF5.txt


['Rv2233']

In [13]:
# Looking at sequence information saved per gene
my_gempro.genes.get_by_id('Rv1295').annotation['sequence']

{'kegg': {'gene': 'Rv1295',
  'ident': '',
  'kegg': 'mtu:Rv1295',
  'metadata_file': 'mtu-Rv1295.kegg',
  'pdbs': [],
  'refseq': None,
  'seq_file': 'mtu-Rv1295.faa',
  'seq_len': 360,
  'uniprot': None},
 'representative': {'kegg_id': ['mtu:Rv1295'],
  'metadata_file': 'P9WG59.txt',
  'pdbs': ['2D1F'],
  'properties': {},
  'seq_file': 'Rv1295.faa',
  'seq_len': 360,
  'uniprot_acc': 'P9WG59'},
 'uniprot': {'P9WG59': {'description': ['TS', 'Threonine synthase'],
   'ec_number': ['4.2.3.1'],
   'entry_version': '2016-10-05',
   'gene': 'Rv1295',
   'gene_name': 'thrC',
   'kegg_id': ['mtu:Rv1295'],
   'metadata_file': 'P9WG59.txt',
   'pdbs': ['2D1F'],
   'pfam': ['PF00291'],
   'refseq': ['NP_215811.1', 'NC_000962.3', 'WP_003406652.1', 'NZ_KK339370.1'],
   'reviewed': True,
   'seq_file': 'P9WG59.fasta',
   'seq_len': 360,
   'seq_version': '2014-04-16',
   'uniprot_acc': 'P9WG59'}}}

## Mapping representative sequence to structure

These are the ways to map sequence to structure:

1. Use the UniProt ID and their automatic mappings to the PDB
2. BLAST the sequence to the PDB
3. Make homology models or 
4. Map to existing homology models

You can only utilize option #1 to map to PDBs if there is a mapped UniProt ID set in the representative sequence. If not, you'll have to BLAST your sequence to the PDB or make a homology model. You can also run both for maximum coverage.

---------------------

### 1. Use the UniProt ID and their automatic mappings to the PDB


- Method:
    - Use the PDBe REST service to query for the best PDB structures for a UniProt ID.
    - Here is the ranking algorithm described by the PDB paper: https://nar.oxfordjournals.org/content/44/D1/D385.full
    - More information found here: https://www.ebi.ac.uk/pdbe/api/doc/sifts.html
    - Link used to retrieve results: https://www.ebi.ac.uk/pdbe/api/mappings/best_structures/:accession
    - The list of PDB structures mapping to a UniProt accession sorted by coverage of the protein and, if the same, resolution.


- Files created:
    - Saves a .json file directly from the web request in the GEM-PRO "sequences" folder
    - No PDBs are downloaded yet
    
    
- Usage:
        my_gempro.map_uniprot_to_pdb(seq_ident_cutoff=0, force_rerun=False)
        
        
- Arguments:

    - *seq_ident_cutoff*
        - From 0 to 1
        - Provide the seq_ident_cutoff as a percentage to filter for structures with only a percent identity above the cutoff.
        - **Warning:** if you set the seq_ident_cutoff too high you risk filtering out PDBs that do match the sequence, but are just missing large portions of it.
        
    - *force_rerun*
        - If you want to force the rerun of mapping, set this to True.
        
        
- Creates attributes:

    - A summary of the rankings is available in the "df_pdb_ranking" attribute.
            my_gempro.df_pdb_ranking
    
        
---------------------------


### 2. BLAST the sequence to the PDB

This will BLAST the representative sequence against the entire PDB, and return significant hits. This will however return hits in other organisms, which may not be ideal.


- Method:
    - BLAST the representative sequence to the entire PDB.


- Files created:
    - Saves a .xml file directly from the web request in the GEM-PRO "sequences" folder
    - No PDBs are downloaded yet
    
    
- Usage:
        my_gempro.blast_seqs_to_pdb(seq_ident_cutoff=0, evalue=0.0001, all_genes=False, force_rerun=False, display_link=False)
        
        
- Arguments:

    - *seq_ident_cutoff*
        - From 0 to 1
        - Provide the seq_ident_cutoff as a percentage to filter for structures with only a percent identity above the cutoff.
        - **Warning:** if you set the seq_ident_cutoff too high you risk filtering out PDBs that do match the sequence, but are just missing large portions of it.
        
    - *evalue*
        - Cutoff for the E-value - filters for significant hits.
        - 0.001 is liberal, 0.0001 is stringent (default).
        
    - *all_genes*
        - Set to True if you want to BLAST all gene sequences
        - Set to False if you only want to BLAST genes without any PDBs (if map_uniprot_to_pdb was already run)
        
    - *force_rerun*
        - If you want to force the rerun of mapping, set this to True.
        
    - *display_link*
        - Display a link to the HTML results of the BLAST result per gene.
        
        
- Creates attributes:

    - A summary of the rankings is available in the "df_pdb_blast" attribute.
            my_gempro.df_pdb_blast


---------------------------


### 3. Make homology models

This will prepare sequences for homology modeling using I-TASSER or allow you to organize already generated ones.

**IN PROGRESS**


- Method:
    - Prepare representative sequences for I-TASSER runs on your local machine, or using Torque (qsub, available on ssb0-ssb4) or SLURM (sbatch, currently used on NERSC) job scheduler systems.


- Files created:
    - Creates homology modeling files in a specified directory
    
    
- Usage:
        my_gempro.prep_itasser_models(outdir, itasser_installation, itlib_location, runtype, print_exec=False, **kwargs)
        
        
- Arguments:

    - *outdir*
        - outdir
        

-------------------


### 4. Map to existing homology models

This will organize the homology models generated by I-TASSER

**IN PROGRESS**


- Method:
    - Prepare representative sequences for I-TASSER runs on your local machine, or using Torque (qsub, available on ssb0-ssb4) or SLURM (sbatch, currently used on NERSC) job scheduler systems.


- Files created:
    - Copies homology models and a couple summary result files into the GEM-PRO "structures" directory
    
    
- Usage:
        my_gempro.get_itasser_models(homology_raw_dir, custom_itasser_name_mapping=None)
        
        
- Arguments:

    - *homology_raw_dir*
        - 
        
    - *custom_itasser_name_mapping*
        - 
        
- Creates attributes:

    - A summary of the I-TASSER modeling is available in the "df_itasser" attribute.
            my_gempro.df_itasser
            

-------------------


### Gene annotations

Check out the gene annotations saved directly into the gene.

- Usage:
        my_gempro.genes.get_by_id('Rv1295').annotation['structure']['pdb']
        my_gempro.genes.get_by_id('Rv1295').annotation['structure']['homology']


In [14]:
# Mapping using the PDBe best_structures service
my_gempro.map_uniprot_to_pdb()

Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"
INFO:ssbio.pipeline.gempro:Completed UniProt -> best PDB mapping. See the "df_pdb_ranking" attribute.


In [15]:
my_gempro.df_pdb_ranking.head()

,gene,uniprot_acc,pdb_id,pdb_chain_id,experimental_method,resolution,seq_coverage,taxonomy_id,pdb_start,pdb_end,unp_start,unp_end,rank
0,Rv1295,P9WG59,2d1f,A,X-ray diffraction,2.50,1.000,1773,1,360,1,360,1
1,Rv1295,P9WG59,2d1f,B,X-ray diffraction,2.50,1.000,1773,1,360,1,360,2
2,Rv1201c,P9WP21,3fsy,A,X-ray diffraction,1.97,0.997,83332,4,319,2,317,1
3,Rv1201c,P9WP21,3fsy,B,X-ray diffraction,1.97,0.997,83332,4,319,2,317,2
4,Rv1201c,P9WP21,3fsy,C,X-ray diffraction,1.97,0.997,83332,4,319,2,317,3


In [16]:
# Mapping using BLAST
my_gempro.blast_seqs_to_pdb(seq_ident_cutoff=.99, all_genes=True)

Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"
INFO:ssbio.pipeline.gempro:Rv3846: Adding 20 PDBs from BLAST results.
INFO:ssbio.pipeline.gempro:Rv2539c: Adding 4 PDBs from BLAST results.
INFO:ssbio.pipeline.gempro:Rv1603: Adding 2 PDBs from BLAST results.
INFO:ssbio.pipeline.gempro:Rv3628: Adding 1 PDBs from BLAST results.
INFO:ssbio.pipeline.gempro:Rv2764c: Adding 1 PDBs from BLAST results.
INFO:ssbio.pipeline.gempro:Rv0467: Adding 4 PDBs from BLAST results.
INFO:ssbio.pipeline.gempro:Rv1415: Adding 2 PDBs from BLAST results.
INFO:ssbio.pipeline.gempro:Rv2220: Adding 24 PDBs from BLAST results.
INFO:ssbio.pipeline.gempro:Rv0253: Adding 1 PDBs from BLAST results.
INFO:ssbio.pipeline.gempro:Rv0548c: Adding 1 PDBs from BLAST results.
INFO:ssbio.pipeline.gempro:Rv2043c: Adding 1 PDBs from BLAST results.
INFO:ssbio.pipeline.gempro:Rv2445c: Adding 1 P

In [17]:
my_gempro.df_pdb_blast.head()

,gene,pdb_id,pdb_chain_id,blast_score,blast_evalue,seq_coverage,seq_similar,seq_num_coverage,seq_num_similar
0,Rv3465,1upi,A,1065.0,3.108690e-116,1.0,1.0,202,202
1,Rv3465,2ixc,A,1065.0,3.582440e-116,1.0,1.0,202,202
2,Rv3465,2ixc,B,1065.0,3.582440e-116,1.0,1.0,202,202
3,Rv3465,2ixc,C,1065.0,3.582440e-116,1.0,1.0,202,202
4,Rv3465,2ixc,D,1065.0,3.582440e-116,1.0,1.0,202,202


In [18]:
# Organizing homology models (only specific to this example)
old_gene_to_homology = pd.read_csv(op.join(my_gempro.data_dir,'161031-old_gene_to_uniprot_mapping.csv'))
gene_to_uniprot = old_gene_to_homology.set_index('m_gene').to_dict()['u_uniprot_acc']

my_gempro.get_itasser_models(homology_raw_dir='/home/nathan/projects_archive/homology_models/MTUBERCULOSIS/raw', custom_itasser_name_mapping=gene_to_uniprot)

Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"
INFO:ssbio.pipeline.gempro:Completed copying of 428 I-TASSER models to GEM-PRO directory. See the "df_itasser" attribute.


In [19]:
my_gempro.df_itasser.head()

,gene,model_file,model_date,difficulty,top_template_pdb,top_template_chain,c_score,tm_score,tm_score_err,rmsd,rmsd_err
0,Rv0417,Rv0417_model1.pdb,2015-12-30,easy,2htm,C,1.66,0.95,0.05,2.6,1.9
1,Rv2291,Rv2291_model1.pdb,2016-01-04,easy,3olh,A,1.38,0.91,0.06,3.3,2.3
2,Rv1559,Rv1559_model1.pdb,2016-01-08,easy,1tdj,A,0.73,0.81,0.09,5.4,3.4
3,Rv3113,Rv3113_model1.pdb,2015-12-30,easy,3sd7,A,0.72,0.81,0.09,4.1,2.8
4,Rv2447c,Rv2447c_model1.pdb,2016-01-08,easy,2vos,A,0.07,0.72,0.11,7.1,4.2


## Setting a representative structure

Once you've mapped PDBs and homology models, then you need to choose between all of the available ones. 


- Method:
    - 


- Files downloaded:
    - 
    
    
- Usage:
        my_gempro.set_representative_structure(always_use_homology=True, 
                                               sort_homology_by='seq_coverage',
                                               allow_missing_on_termini=0.1, 
                                               allow_mutants=True, 
                                               allow_deletions=False,
                                               allow_insertions=False, 
                                               allow_unresolved=True, 
                                               force_rerun=False)
        
        
- Arguments:

    - *always_use_homology*
        - 
        
    - *sort_homology_by*
        - 
            

-------------------


### Gene annotations

Check out the gene annotations saved directly into the gene.

- Usage:
        my_gempro.genes.get_by_id('Rv1295').annotation['structure']['representative']


In [20]:
my_gempro.set_representative_structure()

Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"


## Property calculations

In [21]:
my_gempro.calculate_sequence_properties()
my_gempro.df_sequence_properties.head()

Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"
INFO:ssbio.pipeline.gempro:Created sequence property dataframe. See the "df_sequence_properties" attribute.


,gene,percent_acidic,percent_aliphatic,percent_aromatic,percent_basic,percent_charged,percent_non-polar,percent_polar,percent_small,percent_tiny
0,Rv0417,0.10714,0.44048,0.03571,0.08333,0.19048,0.65079,0.34921,0.62698,0.40476
1,Rv2291,0.10563,0.37324,0.08451,0.09859,0.20423,0.63028,0.36972,0.63028,0.38028
2,Rv3737,0.07561,0.41777,0.05482,0.09074,0.16635,0.64839,0.35161,0.61059,0.40832
3,Rv1295,0.09167,0.38333,0.06944,0.09722,0.18889,0.62500,0.37500,0.61944,0.38333
4,Rv1559,0.13054,0.38228,0.07226,0.09091,0.22145,0.61538,0.38462,0.59207,0.36131


In [22]:
my_gempro.calculate_residue_depth()

Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"


163/|/ 25%|| 163/661 [01:40<05:05,  1.63it/s]

189/|/ 29%|| 189/661 [04:40<11:40,  1.48s/it]

382/|/ 58%|| 382/661 [27:41<20:13,  4.35s/it]

448/|/ 68%|| 448/661 [34:21<16:20,  4.60s/it]

...

565/|/ 85%|| 565/661 [43:22<07:22,  4.61s/it]

619/|/ 94%|| 619/661 [48:42<03:18,  4.72s/it]

633/|/ 96%|| 633/661 [51:42<02:17,  4.90s/it]

645/|/ 98%|| 645/661 [52:42<01:18,  4.90s/it]

INFO:ssbio.pipeline.gempro:Completed calculations of residue depth


In [23]:
my_gempro.genes.get_by_id('Rv1295').annotation['sequence']['representative']['properties']

{'percent_acidic': 0.09167,
 'percent_aliphatic': 0.38333,
 'percent_aromatic': 0.06944,
 'percent_basic': 0.09722,
 'percent_charged': 0.18889,
 'percent_non-polar': 0.625,
 'percent_polar': 0.375,
 'percent_small': 0.61944,
 'percent_tiny': 0.38333}

In [24]:
my_gempro.genes.get_by_id('Rv1295').annotation['structure']['representative']['properties']

{}

## Saving your GEM-PRO

In [25]:
import cobra.io
cobra.io.save_json_model(my_gempro.model, op.join(my_gempro.model_dir, 'iNJ661_GP.json'))